In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from shapely.geometry import Polygon, Point, LineString
from shapely.ops import cascaded_union, unary_union, nearest_points, transform
from shapely.geometry import MultiPoint
import plotly
import plotly.graph_objects as go



c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [9]:
def get_clustered_points(df, n_clusters):
    # Initialize the list of MultiPoints and associated data
    pill_data_list = []
    wellpath_data = []

    # Define the feature matrix for clustering
    X = df.apply(lambda row: [row['geometry'].centroid.x, row['geometry'].centroid.y, -row['Depth']], axis=1).tolist()

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

    # Add cluster labels to the DataFrame
    df['cluster_id'] = kmeans.labels_

    # Calculate the overall mean x and y coordinates of all cavern centroids
    overall_mean_x = df['geometry'].apply(lambda g: g.centroid.x).mean()
    overall_mean_y = df['geometry'].apply(lambda g: g.centroid.y).mean()

    # Calculate the mean x and y coordinates for each cluster
    cluster_means = df.groupby('cluster_id')['geometry'].apply(lambda g: (g.apply(lambda g: g.centroid.x).mean(), g.apply(lambda g: g.centroid.y).mean())).to_dict()

    for index, row in df.iterrows():
        # Initialize the list of points for this pill shape and well path
        pill_points = []
        wellpath_points = []

        # Pill parameters
        r = row['cavern_diameter'] / 2
        H = row['max_cavern_height']
        x_center, y_center = row['geometry'].centroid.x, row['geometry'].centroid.y
        z_center = -row['cav_mid_depth']  # make depth negative

        # Cylinder part of the pill
        z_cylinder = np.linspace(z_center - (H / 2) + r, z_center + (H / 2) - r, 100)
        for z in z_cylinder:
            theta = np.linspace(0, 2 * np.pi, 100)
            x = r * np.cos(theta) + x_center
            y = r * np.sin(theta) + y_center
            for xi, yi in zip(x, y):
                pill_points.append(Point(xi, yi, z))

        # Hemispherical ends of the pill
        phi = np.linspace(0, np.pi, 50)
        theta = np.linspace(0, 2 * np.pi, 50)
        phi, theta = np.meshgrid(phi, theta)

        # Top hemisphere
        x_top = r * np.sin(phi) * np.cos(theta) + x_center
        y_top = r * np.sin(phi) * np.sin(theta) + y_center
        z_top = r * np.cos(phi) + z_center + (H / 2) - r

        # Bottom hemisphere
        x_bottom = r * np.sin(phi) * np.cos(theta) + x_center
        y_bottom = r * np.sin(phi) * np.sin(theta) + y_center
        z_bottom = -(r * np.cos(phi)) + z_center - (H / 2) + r

        for xi, yi, zi in zip(np.concatenate([x_top.flatten(), x_bottom.flatten()]),
                              np.concatenate([y_top.flatten(), y_bottom.flatten()]),
                              np.concatenate([z_top.flatten(), z_bottom.flatten()])):
            pill_points.append(Point(xi, yi, zi))

        # Define control points for well path
        well_top_z = z_center + (H / 2)
        dev_point_z = -10

        # Calculate well path - vertical segment
        x_path_up = np.full(100, x_center)
        y_path_up = np.full(100, y_center)
        z_path_up = np.linspace(well_top_z, dev_point_z, 100)

        # Calculate well path - horizontal segment
        cluster_mean_x, cluster_mean_y = cluster_means[row['cluster_id']]
        x_path_horizontal = np.full(100, cluster_mean_x)
        y_path_horizontal = np.full(100, cluster_mean_y)
        z_path_horizontal = np.full(100, dev_point_z)

        # Concatenate the segments to get the full well path
        x_path = np.concatenate([x_path_up, x_path_horizontal])
        y_path = np.concatenate([y_path_up, y_path_horizontal])
        z_path = np.concatenate([z_path_up, z_path_horizontal])

        # Add well path points to the list
        for x, y, z in zip(x_path, y_path, z_path):
            wellpath_points.append(Point(x, y, z))

        # Add the MultiPoint and its associated data to the list
        pill_data = {
            'geometry': MultiPoint(pill_points),
            'act_cav_vol': row['act_cav_vol'],
            'H2_E_GWh_TM': row['H2_E_GWh_TM'],
            'cav_mid_temp': row['cav_mid_temp'],
            'cavern_diameter': row['cavern_diameter'],
            'obp': row['obp']
        }
        wellpath_data.append({
            'geometry': MultiPoint(wellpath_points),
            'act_cav_vol': row['act_cav_vol'],
            'H2_E_GWh_TM': row['H2_E_GWh_TM'],
            'cav_mid_temp': row['cav_mid_temp'],
            'cavern_diameter': row['cavern_diameter'],
            'obp': row['obp']
        })

        # Create a new GeoDataFrame with the pill geometries
        pill_gdf = gpd.GeoDataFrame([pill_data])
        pill_gdf.set_crs(epsg=23031, inplace=True)
        wellpath_gdf = gpd.GeoDataFrame(wellpath_data)
        wellpath_gdf.set_crs(epsg=23031, inplace=True)

    return df, pill_gdf, wellpath_gdf

In [5]:
def plot_caverns(df, ellipsoid_gdf, wellpath_gdf):
    # Calculate the overall mean x and y coordinates of all cavern centroids
    overall_mean_x = df['geometry'].apply(lambda g: g.centroid.x).mean()
    overall_mean_y = df['geometry'].apply(lambda g: g.centroid.y).mean()

    # Calculate the mean x and y coordinates for each cluster
    cluster_means = df.groupby('cluster_id')['geometry'].apply(lambda g: (g.apply(lambda g: g.centroid.x).mean(), g.apply(lambda g: g.centroid.y).mean())).to_dict()

    # Define a set of colors to cycle through
    color_cycle = plotly.colors.qualitative.Plotly
    color_mapping = {}

    fig = go.Figure()

    for index, row in ellipsoid_gdf.iterrows():
        # Get a unique color for this cavern
        if index not in color_mapping:
            color_mapping[index] = color_cycle[len(color_mapping) % len(color_cycle)]
        
        # Extract points of the ellipsoid and create the scatter3d surface trace
        x_ellipsoid = [point.x for point in row['geometry'].geoms]
        y_ellipsoid = [point.y for point in row['geometry'].geoms]
        z_ellipsoid = [point.z for point in row['geometry'].geoms]
        fig.add_trace(go.Surface(x=x_ellipsoid, y=y_ellipsoid, z=z_ellipsoid, colorscale='Viridis', showscale=False))

        # Extract well path points from the wellpath_gdf
        x_path = [point.x for point in wellpath_gdf.loc[index, 'geometry'].geoms]
        y_path = [point.y for point in wellpath_gdf.loc[index, 'geometry'].geoms]
        z_path = [point.z for point in wellpath_gdf.loc[index, 'geometry'].geoms]

        # Add deviated well path
        well_path = go.Scatter3d(
            x=x_path,
            y=y_path,
            z=z_path,
            mode='lines',
            line=dict(color=color_mapping[index], width=2)  # Use the color for this cavern
        )
        fig.add_trace(well_path)

        # Add line from cluster's mean x/y point at z=0 to the overall mean x/y point at z=0
        cluster_mean_x, cluster_mean_y = cluster_means[df.loc[index, 'cluster_id']]
        fig.add_trace(go.Scatter3d(
            x=[cluster_mean_x, overall_mean_x],
            y=[cluster_mean_y, overall_mean_y],
            z=[0, 0],
            mode='lines',
            line=dict(color='black', width=2)
        ))

    # Update layout properties
    fig.update_layout(
        autosize=True,
        width=1200,
        height=1200,
        scene=dict(
            aspectmode='data',
            zaxis=dict(title='Depth (m)')
        )
    )

    fig.show()


In [4]:
# example how to use above function 
#df, ellipsoid, wellpath_gdf  = get_clustered_points(processed_cavern_data2, 15)
#ellipsoid.to_file("E:\\ellipsoid_paths7.shp")

In [2]:
gdf = gpd.read_file('E:/Paper2OUTPUTMODELS/Bedded/VarCav_MyDat/P50_VarCav_MyDat.gpkg')


In [7]:
gdf

,X,Y,thickness,Depth,Base,thick_avail,cav_top,def_hanging_wall,height_to_diameter_ratio,max_cavern_height,...,Z_Topstass,gfg,cav_mid_temp,insolubles%,act_cav_vol,obp4z2,obp,H2_Mass_ton_TM,H2_E_GWh_TM,geometry
0,294054.5422,6.068508e+06,437.377784,1587.834885,2025.212668,437.377784,1587.834885,0.0,2.248533,307.471717,...,1587.834885,42.845623,75.790453,0.982168,3.777524e+06,0.549807,38.469272,60645.327194,2020.837070,"POLYGON ((294328.029 6068507.619, 294326.712 6..."
1,294104.5422,6.069058e+06,430.522755,1589.491023,2020.013778,430.522755,1589.491023,0.0,2.166273,299.277319,...,1589.491023,35.674862,63.028931,0.934138,3.545936e+06,0.555476,38.542620,59201.974722,1972.741358,"POLYGON ((294380.848 6069057.619, 294379.518 6..."
2,294154.5422,6.069658e+06,414.317427,1599.563860,2013.881287,414.317427,1599.563860,0.0,1.971809,279.605836,...,1599.563860,49.362505,87.259439,0.926743,3.400426e+06,0.570146,38.828347,53346.703784,1777.630718,"POLYGON ((294438.146 6069657.619, 294436.780 6..."
3,294154.5422,6.070258e+06,388.012122,1603.363395,1991.375517,388.012122,1603.363395,0.0,1.740061,250.984906,...,1603.363395,49.686165,87.333555,0.889793,2.950111e+06,0.579946,38.939383,46404.866628,1546.313278,"POLYGON ((294443.021 6070257.619, 294441.631 6..."
4,294154.5422,6.071758e+06,563.837567,1619.350249,2183.187816,563.837567,1619.350249,0.0,4.149076,458.789973,...,1619.350249,46.526408,87.044420,0.927449,3.757910e+06,0.444598,39.169172,59508.016308,1982.939343,"POLYGON ((294375.695 6071757.619, 294374.630 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,302904.5422,6.084608e+06,297.558774,1690.439436,1987.998211,297.558774,1690.439436,0.0,1.287794,171.237561,...,1690.439436,36.239819,65.327788,0.860051,1.515759e+06,0.534635,40.604349,26479.255292,882.347629,"POLYGON ((303170.482 6084607.619, 303169.201 6..."
376,303054.5422,6.074008e+06,300.322100,1699.866040,2000.188140,300.322100,1699.866040,0.0,1.301611,173.610134,...,1699.866040,38.937403,70.607035,0.986690,1.780542e+06,0.536289,41.245331,31110.405660,1036.667851,"POLYGON ((303321.304 6074007.619, 303320.020 6..."
377,303304.5422,6.083458e+06,231.777558,1693.546182,1925.323740,231.777558,1693.546182,0.0,0.958888,116.428357,...,1693.546182,33.573004,59.627146,0.942583,8.471472e+05,0.488197,40.692287,15085.303598,502.675839,"POLYGON ((303547.383 6083457.619, 303546.213 6..."
378,303454.5422,6.084008e+06,256.813254,1699.567401,1956.380655,256.813254,1699.567401,0.0,1.084066,136.869969,...,1699.567401,35.459131,63.587215,0.840492,9.973925e+05,0.507641,40.872150,17629.367296,587.449695,"POLYGON ((303707.054 6084007.619, 303705.838 6..."


In [10]:
get_clustered_points(gdf, 1)

c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


(               X             Y   thickness        Depth         Base  \
 0    294054.5422  6.068508e+06  437.377784  1587.834885  2025.212668   
 1    294104.5422  6.069058e+06  430.522755  1589.491023  2020.013778   
 2    294154.5422  6.069658e+06  414.317427  1599.563860  2013.881287   
 3    294154.5422  6.070258e+06  388.012122  1603.363395  1991.375517   
 4    294154.5422  6.071758e+06  563.837567  1619.350249  2183.187816   
 ..           ...           ...         ...          ...          ...   
 375  302904.5422  6.084608e+06  297.558774  1690.439436  1987.998211   
 376  303054.5422  6.074008e+06  300.322100  1699.866040  2000.188140   
 377  303304.5422  6.083458e+06  231.777558  1693.546182  1925.323740   
 378  303454.5422  6.084008e+06  256.813254  1699.567401  1956.380655   
 379  303804.5422  6.083458e+06  250.110729  1695.484571  1945.595301   
 
      thick_avail      cav_top  def_hanging_wall  height_to_diameter_ratio  \
 0     437.377784  1587.834885              

In [14]:
gdf

,X,Y,thickness,Depth,Base,thick_avail,cav_top,def_hanging_wall,height_to_diameter_ratio,max_cavern_height,...,gfg,cav_mid_temp,insolubles%,act_cav_vol,obp4z2,obp,H2_Mass_ton_TM,H2_E_GWh_TM,geometry,cluster_id
0,294054.5422,6.068508e+06,437.377784,1587.834885,2025.212668,437.377784,1587.834885,0.0,2.248533,307.471717,...,42.845623,75.790453,0.982168,3.777524e+06,0.549807,38.469272,60645.327194,2020.837070,"POLYGON ((294328.029 6068507.619, 294326.712 6...",14
1,294104.5422,6.069058e+06,430.522755,1589.491023,2020.013778,430.522755,1589.491023,0.0,2.166273,299.277319,...,35.674862,63.028931,0.934138,3.545936e+06,0.555476,38.542620,59201.974722,1972.741358,"POLYGON ((294380.848 6069057.619, 294379.518 6...",14
2,294154.5422,6.069658e+06,414.317427,1599.563860,2013.881287,414.317427,1599.563860,0.0,1.971809,279.605836,...,49.362505,87.259439,0.926743,3.400426e+06,0.570146,38.828347,53346.703784,1777.630718,"POLYGON ((294438.146 6069657.619, 294436.780 6...",14
3,294154.5422,6.070258e+06,388.012122,1603.363395,1991.375517,388.012122,1603.363395,0.0,1.740061,250.984906,...,49.686165,87.333555,0.889793,2.950111e+06,0.579946,38.939383,46404.866628,1546.313278,"POLYGON ((294443.021 6070257.619, 294441.631 6...",14
4,294154.5422,6.071758e+06,563.837567,1619.350249,2183.187816,563.837567,1619.350249,0.0,4.149076,458.789973,...,46.526408,87.044420,0.927449,3.757910e+06,0.444598,39.169172,59508.016308,1982.939343,"POLYGON ((294375.695 6071757.619, 294374.630 6...",14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,302904.5422,6.084608e+06,297.558774,1690.439436,1987.998211,297.558774,1690.439436,0.0,1.287794,171.237561,...,36.239819,65.327788,0.860051,1.515759e+06,0.534635,40.604349,26479.255292,882.347629,"POLYGON ((303170.482 6084607.619, 303169.201 6...",0
376,303054.5422,6.074008e+06,300.322100,1699.866040,2000.188140,300.322100,1699.866040,0.0,1.301611,173.610134,...,38.937403,70.607035,0.986690,1.780542e+06,0.536289,41.245331,31110.405660,1036.667851,"POLYGON ((303321.304 6074007.619, 303320.020 6...",4
377,303304.5422,6.083458e+06,231.777558,1693.546182,1925.323740,231.777558,1693.546182,0.0,0.958888,116.428357,...,33.573004,59.627146,0.942583,8.471472e+05,0.488197,40.692287,15085.303598,502.675839,"POLYGON ((303547.383 6083457.619, 303546.213 6...",0
378,303454.5422,6.084008e+06,256.813254,1699.567401,1956.380655,256.813254,1699.567401,0.0,1.084066,136.869969,...,35.459131,63.587215,0.840492,9.973925e+05,0.507641,40.872150,17629.367296,587.449695,"POLYGON ((303707.054 6084007.619, 303705.838 6...",0


In [15]:
df, ellipsoid, wellpath_gdf  = get_clustered_points(gdf, 1)

c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [16]:
#plot_caverns(df, ellipsoid, wellpath_gdf)

In [13]:
ellipsoid

,geometry,act_cav_vol,H2_E_GWh_TM,cav_mid_temp,cavern_diameter,obp
0,MULTIPOINT Z (303867.053 6083457.619 -1789.319...,1.036250e+06,608.77351,64.205341,125.020756,40.842364


In [18]:
ellipsoid.to_file("E:\p50_multiz caverns_structure7.shp")

C:\Users\c0063859\AppData\Local\Temp\ipykernel_22284\4081318894.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ellipsoid.to_file("E:\p50_multiz caverns_structure7.shp")


In [1]:
import numpy as np
from shapely.geometry import Point, MultiPoint
import geopandas as gpd

def get_pill_shapes(df):
    # Initialize the list of MultiPoints for pill shapes and well paths
    pill_data_list = []
    wellpath_data = []

    for index, row in df.iterrows():
        # Initialize the list of points for this pill shape and well path
        pill_points = []

        # Pill parameters
        r = row['cavern_diameter'] / 2
        H = row['max_cavern_height']
        x_center, y_center = row['geometry'].centroid.x, row['geometry'].centroid.y
        z_center = -row['cav_mid_depth']  # make depth negative

        # Cylinder part of the pill
        z_cylinder = np.linspace(z_center - (H / 2) + r, z_center + (H / 2) - r, 100)
        for z in z_cylinder:
            theta = np.linspace(0, 2 * np.pi, 100)
            x = r * np.cos(theta) + x_center
            y = r * np.sin(theta) + y_center
            for xi, yi in zip(x, y):
                pill_points.append(Point(xi, yi, z))

        # Hemispherical ends of the pill
        phi = np.linspace(0, np.pi, 50)
        theta = np.linspace(0, 2 * np.pi, 50)
        phi, theta = np.meshgrid(phi, theta)

        # Top hemisphere
        x_top = r * np.sin(phi) * np.cos(theta) + x_center
        y_top = r * np.sin(phi) * np.sin(theta) + y_center
        z_top = r * np.cos(phi) + z_center + (H / 2) - r

        # Bottom hemisphere
        x_bottom = r * np.sin(phi) * np.cos(theta) + x_center
        y_bottom = r * np.sin(phi) * np.sin(theta) + y_center
        z_bottom = -(r * np.cos(phi)) + z_center - (H / 2) + r

        for xi, yi, zi in zip(np.concatenate([x_top.flatten(), x_bottom.flatten()]),
                              np.concatenate([y_top.flatten(), y_bottom.flatten()]),
                              np.concatenate([z_top.flatten(), z_bottom.flatten()])):
            pill_points.append(Point(xi, yi, zi))

        # Append the MultiPoint geometry of the pill shape to the list
        pill_data = {
            'geometry': MultiPoint(pill_points),
            'act_cav_vol': row['act_cav_vol'],
            'H2_E_GWh_TM': row['H2_E_GWh_TM'],
            'cav_mid_temp': row['cav_mid_temp'],
            'cavern_diameter': row['cavern_diameter'],
            'obp': row['obp']
        }
        pill_data_list.append(pill_data)

    # Create a GeoDataFrame with the pill geometries and return it
    pill_gdf = gpd.GeoDataFrame(pill_data_list)
    pill_gdf.set_crs(epsg=23031, inplace=True)

    return pill_gdf


c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
c:\ProgramData\Anaconda3\envs\Geol\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [13]:
import numpy as np
from shapely.geometry import Point, MultiPoint
import geopandas as gpd
from collections import Counter

def check_for_duplicate_points(multi_point_obj):
    point_counter = {}
    for point in multi_point_obj.geoms:
        point_tuple = tuple(point.coords[0])
        if point_tuple in point_counter:
            point_counter[point_tuple] += 1
        else:
            point_counter[point_tuple] = 1
    duplicates = [(Point(coords), count) for coords, count in point_counter.items() if count > 1]
    return duplicates

def get_pill_shapes(df):
    pill_data_list = []
    geometry_count = {}
    
    for index, row in df.iterrows():
        pill_points = []
        
        # Pill parameters
        r = row['cavern_diameter'] / 2
        H = row['max_cavern_height']
        x_center, y_center = row['geometry'].centroid.x, row['geometry'].centroid.y
        z_center = -row['cav_mid_depth']  # make depth negative

        # Debug: Print out important parameters
        print(f"Parameters for index {index}: r={r}, H={H}, x_center={x_center}, y_center={y_center}, z_center={z_center}")

        # Cylinder part of the pill
        z_cylinder = np.linspace(z_center - (H / 2) + r, z_center + (H / 2) - r, 100)
        for z in z_cylinder:
            theta = np.linspace(0, 2 * np.pi, 100)
            x = r * np.cos(theta) + x_center
            y = r * np.sin(theta) + y_center
            for xi, yi in zip(x, y):
                pill_points.append(Point(xi, yi, z))

        # Hemispherical ends of the pill
        phi = np.linspace(0, np.pi, 50)
        theta = np.linspace(0, 2 * np.pi, 50)
        phi, theta = np.meshgrid(phi, theta)

# Top hemisphere
        x_top = r * np.sin(phi) * np.cos(theta) + x_center
        y_top = r * np.sin(phi) * np.sin(theta) + y_center
        z_top = r * np.cos(phi) + z_center + (H / 2) - r

# Bottom hemisphere
        x_bottom = r * np.sin(phi) * np.cos(theta) + x_center
        y_bottom = r * np.sin(phi) * np.sin(theta) + y_center
        z_bottom = -r * np.cos(phi) + z_center - (H / 2) + r

# Only keep half of the points for each hemisphere to eliminate redundancy
        x_top, y_top, z_top = x_top[:, :25], y_top[:, :25], z_top[:, :25]
        x_bottom, y_bottom, z_bottom = x_bottom[:, :25], y_bottom[:, :25], z_bottom[:, :25]

# Add points to pill_points
        for xi, yi, zi in zip(np.concatenate([x_top.flatten(), x_bottom.flatten()]),
                            np.concatenate([y_top.flatten(), y_bottom.flatten()]),
                            np.concatenate([z_top.flatten(), z_bottom.flatten()])):
            pill_points.append(Point(xi, yi, zi))


        # Create the MultiPoint geometry of the pill shape
        multi_point_pill = MultiPoint(pill_points)

        # Debug: Check for duplicate points
        duplicates = check_for_duplicate_points(multi_point_pill)
        if duplicates:
            print(f"Duplicate points found for index {index}: {duplicates}")

        # Debug: Update geometry_count
        if index in geometry_count:
            geometry_count[index] += 1
        else:
            geometry_count[index] = 1
        
        pill_data = {
            'geometry': multi_point_pill,
            'act_cav_vol': row['act_cav_vol'],
            'H2_E_GWh_TM': row['H2_E_GWh_TM'],
            'cav_mid_temp': row['cav_mid_temp'],
            'cavern_diameter': row['cavern_diameter'],
            'obp': row['obp']
        }
        pill_data_list.append(pill_data)

    # Debug: Print out the geometry_count
    print("Geometry count for each polygon:", geometry_count)

    pill_gdf = gpd.GeoDataFrame(pill_data_list)
    pill_gdf.set_crs(epsg=23031, inplace=True)
    
    return pill_gdf

# Uncomment the following line to run the function on your data
# get_pill_shapes(df)


In [14]:
Test = get_pill_shapes(gdf)

Parameters for index 0: r=68.37161418865342, H=307.47171660064515, x_center=294054.5422000001, y_center=6068507.618800004, z_center=-1768.9193887184556
Duplicate points found for index 0: [(<POINT Z (294122.914 6068507.619 -1854.284)>, 2), (<POINT Z (294122.914 6068507.619 -1852.559)>, 2), (<POINT Z (294122.914 6068507.619 -1850.835)>, 2), (<POINT Z (294122.914 6068507.619 -1849.11)>, 2), (<POINT Z (294122.914 6068507.619 -1847.386)>, 2), (<POINT Z (294122.914 6068507.619 -1845.661)>, 2), (<POINT Z (294122.914 6068507.619 -1843.936)>, 2), (<POINT Z (294122.914 6068507.619 -1842.212)>, 2), (<POINT Z (294122.914 6068507.619 -1840.487)>, 2), (<POINT Z (294122.914 6068507.619 -1838.763)>, 2), (<POINT Z (294122.914 6068507.619 -1837.038)>, 2), (<POINT Z (294122.914 6068507.619 -1835.314)>, 2), (<POINT Z (294122.914 6068507.619 -1833.589)>, 2), (<POINT Z (294122.914 6068507.619 -1831.865)>, 2), (<POINT Z (294122.914 6068507.619 -1830.14)>, 2), (<POINT Z (294122.914 6068507.619 -1828.416)>, 2

In [15]:
Test.to_file("E:\Multi_Z_Pill_Fix.shp")

C:\Users\c0063859\AppData\Local\Temp\ipykernel_33920\2173339577.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  Test.to_file("E:\Multi_Z_Pill_Fix.shp")


In [ ]:
Test

In [44]:
gdf = gpd.read_file('E:/structure/Structure_zone.shp')

In [41]:
def shrink_square_polygon(gdf, distance):
    """
    Shrink the square or rectangular polygon(s) in a GeoDataFrame inward by a specified distance.
    
    Parameters:
    - gdf: GeoDataFrame containing the polygon geometry
    - distance: float value to shrink the polygon inward
    
    Returns:
    - GeoDataFrame with shrunk polygons
    """
    # Create a deep copy to avoid modifying the original GeoDataFrame
    gdf_copy = gdf.copy()
    
    def shrink_coords(coords, distance):
        new_coords = [(x + distance, y + distance) if i % 2 == 0 else (x - distance, y - distance) for i, (x, y) in enumerate(coords[:-1])]
        new_coords.append(new_coords[0])  # Close the polygon
        return new_coords
    
    # Shrink each polygon
    gdf_copy['geometry'] = gdf_copy['geometry'].apply(lambda x: Polygon(shrink_coords(list(x.exterior.coords), distance)))
    
    # Remove any invalid geometries (like collapsed polygons)
    gdf_copy = gdf_copy.loc[gdf_copy.geometry.is_valid]
    
    return gdf_copy

In [48]:
gdfnew = shrink_polygon(gdf, -250)

In [49]:
gdfnew

,Type,Domain,Droid,Comment,ShapeName,Project,geometry


In [ ]:
# Create a simple polygon
polygon = Polygon([(0, 0), (5, 0), (5, 5), (0, 5), (0, 0)])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': [polygon]})

# Plot original polygon
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

gdf.boundary.plot(ax=ax[0], color='blue')
ax[0].set_title('Original Polygon')

# Shrink the polygon using the function
shrunken_gdf = shrink_polygon(gdf.copy(), -1)

# Plot shrunk polygon
shrunken_gdf.boundary.plot(ax=ax[1], color='red')
ax[1].set_title('Shrunk Polygon')

plt.show()